View this example on [Google Colab](https://colab.research.google.com/drive/1tKpNj_TZz_h8lEki8kgCHI5QBgbv_7q1?usp=sharing)

In [ ]:
import importlib.util
if importlib.util.find_spec('vitessce') is None:
  !pip install vitessce[all]

In [ ]:
from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    MultiImageWrapper,
    OmeTiffWrapper,
)

In [ ]:
vc = VitessceConfig(schema_version="1.0.15", name='Spleen')
dataset = vc.add_dataset(name='HBM287.WDKX.539').add_object(
    AnnDataWrapper(
        adata_url="https://assets.hubmapconsortium.org/69d9c52bc9edb625b496cecb623ec081/anndata-zarr/reg001_expr-anndata.zarr",
        obs_feature_matrix_path="X",
        obs_locations_path="obsm/xy",
        obs_embedding_paths=["obsm/tsne"],
        obs_embedding_names=["t-SNE"],
        obs_set_paths=[
            "obs/Cell K-Means [tSNE_All_Features]",
            "obs/Cell K-Means [Mean-All-SubRegions] Expression",
            "obs/Cell K-Means [Mean] Expression",
        ],
        obs_set_names=[
            "Cell K-Means [tSNE_All_Features]",
            "Cell K-Means [Mean-All-SubRegions] Expression",
            "Cell K-Means [Mean] Expression",
        ]
    )
).add_object(
    MultiImageWrapper([
        OmeTiffWrapper(img_url="https://assets.hubmapconsortium.org/69d9c52bc9edb625b496cecb623ec081/ometiff-pyramids/pipeline_output/mask/reg001_mask.ome.tif?token=", is_bitmask=True, name="Segmentations"),
        OmeTiffWrapper(img_url="https://assets.hubmapconsortium.org/69d9c52bc9edb625b496cecb623ec081/ometiff-pyramids/pipeline_output/expr/reg001_expr.ome.tif?token=", is_bitmask=False, name="Image")
    ])
)
spatial_plot = vc.add_view(cm.SPATIAL, dataset=dataset)
layer_controller = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset)
tsne_plot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="t-SNE")
cell_set_manager = vc.add_view(cm.OBS_SETS, dataset=dataset)
cell_set_sizes = vc.add_view(cm.OBS_SET_SIZES, dataset=dataset)


spatial_segmentation_layer_value = [{
    "type":"bitmask",
    "index":0,
    "visible":True,
    "colormap":None,
    "opacity":1,
    "domainType":"Min/Max",
    "transparentColor":[0,0,0],
    "renderingMode":"Additive",
    "use3d":False,
    "channels":[
        {"selection":{"c":0,"t":0,"z":0},"color":[0,0,0],"visible":False,"slider":[0,1]},
        {"selection":{"c":1,"t":0,"z":0},"color":[0,0,0],"visible":True,"slider":[0,1]}, # Set the nuclei channel as checked initially
        {"selection":{"c":2,"t":0,"z":0},"color":[0,0,0],"visible":False,"slider":[0,1]},
        {"selection":{"c":3,"t":0,"z":0},"color":[0,0,0],"visible":False,"slider":[0,1]}
    ]
}]
vc.link_views([spatial_plot, layer_controller], [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y, ct.SPATIAL_SEGMENTATION_LAYER], [-4, 5000, 5000, spatial_segmentation_layer_value])


vc.layout((spatial_plot | layer_controller) / (tsne_plot | (cell_set_manager | cell_set_sizes)));

In [ ]:
from IPython.display import display, HTML

url = vc.web_app()
display(HTML(f'<a href="{url}" target="_blank">View on Vitessce.io</a>'))

In [ ]:
vw = vc.widget()
vw